## **Importing Libraries**


In [19]:
#Installing the required packages
!pip install librosa
!pip install keras
!pip install tensorflow
!pip install ffmpeg
!pip install noisereduce
!pip install hmmlearn


In [20]:
#Importing the required libraries
import os
import glob
import librosa
import librosa.display
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.io import wavfile
from scipy import fftpack
from scipy.stats import kurtosis,skew,mode
import sklearn.preprocessing,sklearn.decomposition
from sklearn.metrics import accuracy_score,confusion_matrix
from sklearn.utils import shuffle
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm
from sklearn.naive_bayes import MultinomialNB,GaussianNB
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedShuffleSplit,StratifiedKFold,train_test_split
from keras import utils
import keras
from keras import layers
from keras.layers import Activation, Dense, Dropout, Conv1D, Conv2D, Flatten,Reshape, BatchNormalization, ZeroPadding2D,MaxPooling1D,AveragePooling1D, MaxPooling2D, GlobalMaxPooling2D, GlobalAveragePooling1D, AveragePooling2D, Input, Add
from keras.models import Sequential
from keras import regularizers,optimizers
from tensorflow.keras.optimizers import SGD,Adam
from tensorflow.keras.utils import to_categorical
import keras.backend as K
from keras.models import load_model
from keras.callbacks import EarlyStopping
import warnings
warnings.filterwarnings("ignore")

## **Google Drive Link: https://drive.google.com/drive/folders/1rqxai0B95AHMcaJUUfA2CHMF5bCwHPr2?usp=sharing**

In [21]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [22]:
import os
print(os.listdir('/content/drive/MyDrive/HW5_Dataset-main/HW5_Dataset-main/en'))


['train.tsv', 'test.tsv', 'clips', 'train_extracted', 'test_extracted', 'train_extracted_denoised', 'test_extracted_denoised']


## **Extracting Training Data from Common Voice. Saves the processed audio files in numpy format**


In [ ]:
# Function to extract the training data for Common Voice
def get_training(original_path):

    df = pd.read_csv(os.path.join(original_path, 'train.tsv'), sep='\t')

    # Creating a folder to store the Numpy arrays
    extracted_path = os.path.join(original_path, 'train_extracted')
    if not os.path.exists(extracted_path):
        os.makedirs(extracted_path)

    # Getting the file names of audios from the dataframe
    audio_files = np.array(df['path'])

    # Loading each audio file and save it as a numpy array
    for i, audio_file in enumerate(audio_files):
        audio_path = os.path.join(original_path, 'clips', audio_file)
        d, r = librosa.load(audio_path, mono=True)

        # Saving the audio as a numpy array with the same filename (but with .npy extension)
        np.save(os.path.join(extracted_path, audio_file.replace('.mp3', '.npy')), d)

        if i % 1000 == 0:
            print(f'Processed {i}/{len(audio_files)} files')

get_training(r'/content/drive/MyDrive/HW5_Dataset-main/HW5_Dataset-main/en')

## **Extracting Testing Data from Common Voice. Saves the processed audio files in numpy format**


In [ ]:
# Function to extract the testing data for Common Voice
def get_testing(original_path):

    df = pd.read_csv(os.path.join(original_path, 'test.tsv'), sep='\t')

    # Creating a folder to store the Numpy arrays
    extracted_path = os.path.join(original_path, 'test_extracted')
    if not os.path.exists(extracted_path):
        os.makedirs(extracted_path)

    # Getting the file names of audios from the dataframe
    audio_files = np.array(df['path'])

    # Loading each audio file and save it as a numpy array
    for i, audio_file in enumerate(audio_files):
        audio_path = os.path.join(original_path, 'clips', audio_file)
        d, r = librosa.load(audio_path, mono=True)

        # Saving the audio as a numpy array with the same filename (but with .npy extension)
        np.save(os.path.join(extracted_path, audio_file.replace('.mp3', '.npy')), d)

        if i % 1000 == 0:
            print(f'Processed {i}/{len(audio_files)} files')

get_testing(r'/content/drive/MyDrive/HW5_Dataset-main/HW5_Dataset-main/en')


## **Extracting MFCC Features**


In [ ]:
# Function to extract MFCC features which takes a CSV file and the extracted folder as arguments
def get_mfcc_features(original_path, csv_file, extracted_folder):
    # Loading the CSV file into a dataframe.
    df = pd.read_csv(os.path.join(original_path, csv_file), sep='\t')

    # Geting the audio file names from the 'path' column.
    audio_extracted = np.array(df['path'])

    # Creating an empty list to store the features.
    mfcc_features = []

    # Looping on each audio file path.
    for i in range(len(audio_extracted)):
      
        audio_file_data = np.load(os.path.join(original_path, extracted_folder, audio_extracted[i].replace('.mp3', '.npy')))

        # Calculating MFCC coefficients for the audio sequence.
        mfcc_data = librosa.feature.mfcc(y=audio_file_data, sr=22050, n_fft=441, hop_length=220)

        # Calculating various statistic measures on the coefficients.
        mean_mfcc = np.mean(mfcc_data, axis=1)
        median_mfcc = np.median(mfcc_data, axis=1)
        std_mfcc = np.std(mfcc_data, axis=1)
        skew_mfcc = skew(mfcc_data, axis=1)
        kurt_mfcc = kurtosis(mfcc_data, axis=1)
        maximum_mfcc = np.amax(mfcc_data, axis=1)
        minimum_mfcc = np.amin(mfcc_data, axis=1)

        # Concatenating all the statistic measures and adding to the feature list.
        addList = np.concatenate((mean_mfcc, median_mfcc, std_mfcc, skew_mfcc, kurt_mfcc, maximum_mfcc, minimum_mfcc))
        mfcc_features.append(addList)

    return mfcc_features

mfcc_features = get_mfcc_features(original_path=r'/content/drive/MyDrive/HW5_Dataset-main/HW5_Dataset-main/en', csv_file='train.tsv', extracted_folder='train_extracted')


## **Printing Out MFCC Features**

In [ ]:
print(len(mfcc_features))  # Checking how many audio files were processed
print(mfcc_features[0])    # Checking the features of the first audio file

3722
[-3.95441254e+02  8.21870651e+01  2.32697563e+01  1.52611856e+01
  2.73518200e+01 -6.75943518e+00  7.70663261e+00  5.09011269e+00
  1.33283389e+00 -1.02458601e+01  3.35489488e+00  6.89743662e+00
 -1.15976248e+01  2.76556611e-01 -3.87924582e-01 -1.20999069e+01
  9.17172253e-01  2.31625938e+00 -1.27923012e+01  6.54831743e+00
 -3.53572876e+02  9.14794006e+01  2.26850319e+01  1.52714901e+01
  2.39889297e+01 -6.13467598e+00  8.47735882e+00  6.67430305e+00
  3.08753657e+00 -7.10871887e+00  3.98385954e+00  4.82890940e+00
 -1.02689190e+01  8.20303440e-01 -8.65521908e-01 -1.07920074e+01
 -7.37263799e-01  1.74528897e+00 -1.25675049e+01  5.17714214e+00
  1.41427826e+02  9.89742050e+01  3.34292450e+01  2.86911964e+01
  2.96158600e+01  2.29080257e+01  1.35822344e+01  1.31000633e+01
  1.67239742e+01  1.53002558e+01  1.23246040e+01  1.22837448e+01
  1.26071091e+01  1.19653368e+01  1.15504541e+01  1.19527693e+01
  1.17042465e+01  1.06747904e+01  1.09539433e+01  1.04262629e+01
 -5.40725589e-01 -5.

## **Extracting Spectrograms**

In [ ]:
import os
import numpy as np
import pandas as pd
import librosa
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedShuffleSplit
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder

def get_spectrograms(original_path, train_csv, train_extracted, test_csv, test_extracted):

    # Reading the train_csv for Training Dataset file names.
    df_train = pd.read_csv(os.path.join(original_path, train_csv), sep='\t')
    audio_extracted_train = np.array(df_train['path'])
    labels_train = np.array(df_train['accents'])

    # Initializing empty lists for storing spectrograms and labels.
    X_train = []
    Y_train = []
    X_test = []

    # Variable to keep track of the maximum length across both training and testing sets
    max_len = 0

    # Looping through the Training Data Audio sequences.
    for i in range(len(audio_extracted_train)):
        filename = audio_extracted_train[i]
        if filename.endswith('.mp3'):
            filename = filename.replace('.mp3', '')  # Remove .mp3 if it exists
        filename = f"{filename}.npy"  
        file_path = os.path.join(original_path, train_extracted, filename)

        # Checking if the file exists before loading
        if os.path.isfile(file_path):
            audio_file_data = np.load(file_path)
            mel_spec = librosa.power_to_db(librosa.feature.melspectrogram(y=audio_file_data[:650000], n_mels=128, hop_length=2048), ref=np.max)
            X_train.append(mel_spec)
            Y_train.append(labels_train[i])

            # Updating the max_len to the longest spectrogram
            if mel_spec.shape[1] > max_len:
                max_len = mel_spec.shape[1]
        else:
            print(f"File not found: {file_path}")

    # Pading the training spectrograms to make them the same length
    X_train_padded = []
    for spec in X_train:
        padding = max_len - spec.shape[1]
        if padding > 0:
            spec = np.pad(spec, ((0, 0), (0, padding)), mode='constant')
        X_train_padded.append(spec)

    # Reading the test_csv for Testing Dataset file names.
    df_test = pd.read_csv(os.path.join(original_path, test_csv), sep='\t')
    audio_extracted_test = np.array(df_test['path'])

    # Looping through the testing data audio sequences.
    for i in range(len(audio_extracted_test)):
        filename = audio_extracted_test[i]
        if filename.endswith('.mp3'):
            filename = filename.replace('.mp3', '')  # Remove .mp3 if it exists
        filename = f"{filename}.npy" 
        file_path = os.path.join(original_path, test_extracted, filename)

        # Checking if the file exists before loading
        if os.path.isfile(file_path):
            audio_file_data = np.load(file_path)
            mel_spec = librosa.power_to_db(librosa.feature.melspectrogram(y=audio_file_data[:650000], n_mels=128, hop_length=2048), ref=np.max)
            X_test.append(mel_spec)
            # Updating the max_len to the longest spectrogram
            if mel_spec.shape[1] > max_len:
                max_len = mel_spec.shape[1]
        else:
            print(f"File not found: {file_path}")

    # Pading the test spectrograms to have the same shape as training
    X_test_padded = []
    for spec in X_test:
        padding = max_len - spec.shape[1]
        if padding > 0:
            spec = np.pad(spec, ((0, 0), (0, padding)), mode='constant')
        X_test_padded.append(spec)

    # Converting lists into numpy arrays.
    X_train = np.array(X_train_padded)
    Y_train = np.array(Y_train)
    X_test = np.array(X_test_padded)

    # Encoding labels as integers
    label_encoder = LabelEncoder()
    Y_train_encoded = label_encoder.fit_transform(Y_train)

    # Checking the shapes after conversion
    print(f"Shapes after conversion: X_train {X_train.shape}, Y_train {Y_train_encoded.shape}, X_test {X_test.shape}")

    # Splitting the training features into Training and Validation.
    stratified_split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
    for train_index, val_index in stratified_split.split(X_train, Y_train_encoded):
        training_data, train_labels = X_train[train_index], Y_train_encoded[train_index]
        val_data, val_labels = X_train[val_index], Y_train_encoded[val_index]

    # One-hot encoding the Training and Validation labels.
    num_classes = len(np.unique(Y_train_encoded))
    Y_train = to_categorical(train_labels, num_classes)
    Y_val = to_categorical(val_labels, num_classes)

    # Reshaping the features into (rows, columns, 1) for CNNs.
    X_train = np.array([training_data[i].reshape(training_data[i].shape[0], training_data[i].shape[1], 1) for i in range(len(training_data))])
    X_val = np.array([val_data[i].reshape(val_data[i].shape[0], val_data[i].shape[1], 1) for i in range(len(val_data))])
    X_test = np.array([X_test[i].reshape(X_test[i].shape[0], X_test[i].shape[1], 1) for i in range(len(X_test))])

        # Return the features and labels
    return X_train, Y_train, X_val, Y_val, X_test

# Run the function with paths
all_features = get_spectrograms(
    original_path=r'/content/drive/MyDrive/HW5_Dataset-main/HW5_Dataset-main/en',
    train_csv='train.tsv',
    test_csv='test.tsv',
    train_extracted='train_extracted',
    test_extracted='test_extracted'
)

Shapes after conversion: X_train (3722, 128, 112), Y_train (3722,), X_test (2945, 128, 115)


## **Training and Evaluating a Random Forest Model on Processed Data**


In [ ]:
#all_features
X_train, Y_train, X_val, Y_val, X_test = all_features

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score

# Function to use Random Forest classifier
def random_forest(X_train, Y_train, X_test):
    # Initializing Random Forest classifier with number of trees as 200.
    random_forest = RandomForestClassifier(n_estimators=200)

    # Fiting Training Dataset.
    random_forest.fit(X_train, Y_train)

    # Predict and return labels.
    return random_forest.predict(X_test)

# Flatten the data for the Random Forest model
X_train_flat = [x.flatten() for x in X_train]
X_val_flat = [x.flatten() for x in X_val]
X_test_flat = [x.flatten() for x in X_test]

# Converting lists to numpy arrays
X_train_flat = np.array(X_train_flat)
X_val_flat = np.array(X_val_flat)
X_test_flat = np.array(X_test_flat)

# Finding the minimum feature length
min_len = min(X_train_flat.shape[1], X_val_flat.shape[1])

# Truncating both to the minimum length if necessary
X_train_flat = X_train_flat[:, :min_len]
X_val_flat = X_val_flat[:, :min_len]
X_test_flat = X_test_flat[:, :min_len]

# Fit the model and make predictions for the validation set
Y_pred_val = random_forest(X_train_flat, Y_train, X_val_flat)
rf_accuracy = accuracy_score(Y_val, Y_pred_val)

# Evaluating the model's performance on the validation set
print("Accuracy on validation set:", accuracy_score(Y_val, Y_pred_val))
print("Classification Report:\n", classification_report(Y_val, Y_pred_val))

Accuracy on validation set: 0.9489932885906041
Classification Report:
               precision    recall  f1-score   support

           0       0.97      0.76      0.85       131
           1       1.00      0.98      0.99       126
           2       0.95      0.99      0.97       488

   micro avg       0.96      0.95      0.96       745
   macro avg       0.97      0.91      0.94       745
weighted avg       0.96      0.95      0.95       745
 samples avg       0.95      0.95      0.95       745



## **Using Grid Search for Boosting Random Forest Model Performance**


In [ ]:
# Importing necessary libraries
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, accuracy_score
import numpy as np

X_train, Y_train, X_val, Y_val, X_test = all_features

# Flatten the data for the Random Forest model
X_train_flat = np.array([x.flatten() for x in X_train])
X_val_flat = np.array([x.flatten() for x in X_val])
X_test_flat = np.array([x.flatten() for x in X_test])

# Ensuring consistent feature dimensions
min_len = min(X_train_flat.shape[1], X_val_flat.shape[1])
X_train_flat = X_train_flat[:, :min_len]
X_val_flat = X_val_flat[:, :min_len]
X_test_flat = X_test_flat[:, :min_len]

#Default Random Forest for baseline comparison
def random_forest(X_train, Y_train, X_test):
    # Initialize Random Forest classifier with default parameters
    random_forest = RandomForestClassifier(n_estimators=200, random_state=42)
    # Fit training data
    random_forest.fit(X_train, Y_train)
    # Predict and return labels
    return random_forest.predict(X_test)

Y_pred_val = random_forest(X_train_flat, Y_train, X_val_flat)
rf_accuracy = accuracy_score(Y_val, Y_pred_val)

# Print baseline accuracy and classification report
print("Baseline Accuracy on Validation Set:", rf_accuracy)
print("Baseline Classification Report:\n", classification_report(Y_val, Y_pred_val))

#Hyperparameter Tuning with Grid Search
param_grid = {
    'n_estimators': [100, 200, 300, 500],
    'max_depth': [10, 20, 30, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Initializing GridSearchCV with Random Forest
rf_model = RandomForestClassifier(random_state=42)
grid_search = GridSearchCV(estimator=rf_model, param_grid=param_grid, cv=3, scoring='accuracy', verbose=2)

# Fiting Grid Search on training data
print("Starting Grid Search...")
grid_search.fit(X_train_flat, Y_train)

# Best parameters and accuracy from Grid Search
best_params = grid_search.best_params_
best_accuracy = grid_search.best_score_

print("\nBest Parameters from Grid Search:", best_params)
print("Best Cross-Validated Accuracy from Grid Search:", best_accuracy)

#Evaluating Best Model on Validation Set
best_rf_model = grid_search.best_estimator_
Y_pred_val_tuned = best_rf_model.predict(X_val_flat)

# Validation accuracy and classification report with tuned parameters
rf_tuned_accuracy = accuracy_score(Y_val, Y_pred_val_tuned)
print("\nTuned Model Accuracy on Validation Set:", rf_tuned_accuracy)
print("Tuned Model Classification Report:\n", classification_report(Y_val, Y_pred_val_tuned))

#Testing the Best Model on Test Set
Y_pred_test = best_rf_model.predict(X_test_flat)
rf_test_accuracy = accuracy_score(Y_val, Y_pred_val_tuned)
print("\nTest Set Accuracy with Tuned Model:", rf_test_accuracy)

Baseline Accuracy on Validation Set: 0.9530201342281879
Baseline Classification Report:
               precision    recall  f1-score   support

           0       0.95      0.79      0.87       131
           1       1.00      0.98      0.99       126
           2       0.95      0.99      0.97       488

   micro avg       0.96      0.95      0.96       745
   macro avg       0.97      0.92      0.94       745
weighted avg       0.96      0.95      0.96       745
 samples avg       0.95      0.95      0.95       745

Starting Grid Search...
Fitting 3 folds for each of 144 candidates, totalling 432 fits
[CV] END max_depth=10, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=  10.9s
[CV] END max_depth=10, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=  11.3s
[CV] END max_depth=10, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=  10.0s
[CV] END max_depth=10, min_samples_leaf=1, min_samples_split=2, n_estimators=200; tota

## **Training and Evaluating a Decision Tree Model on Processed Data**

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, accuracy_score
import numpy as np

# Function to use Decision Tree classifier
def decision_tree(X_train, Y_train, X_test):
    # Initialize Decision Tree classifier.
    decision_tree = DecisionTreeClassifier()

    # Fiting Training Dataset.
    decision_tree.fit(X_train, Y_train)

    # Predict and return labels.
    return decision_tree.predict(X_test)

# Preparing the data
X_train, Y_train, X_val, Y_val, X_test = all_features

# Flatten the data for the Decision Tree model
X_train_flat = [x.flatten() for x in X_train]
X_val_flat = [x.flatten() for x in X_val]
X_test_flat = [x.flatten() for x in X_test]

# Convert lists to numpy arrays
X_train_flat = np.array(X_train_flat)
X_val_flat = np.array(X_val_flat)
X_test_flat = np.array(X_test_flat)

# Find the minimum feature length
min_len = min(X_train_flat.shape[1], X_val_flat.shape[1])

# Truncate both to the minimum length if necessary
X_train_flat = X_train_flat[:, :min_len]
X_val_flat = X_val_flat[:, :min_len]
X_test_flat = X_test_flat[:, :min_len]

# Fit the model and make predictions for the validation set
Y_pred_val = decision_tree(X_train_flat, Y_train, X_val_flat)
dt_accuracy = accuracy_score(Y_val, Y_pred_val)

# Evaluating the model's performance on the validation set
print("Accuracy on validation set:", accuracy_score(Y_val, Y_pred_val))
print("Classification Report:\n", classification_report(Y_val, Y_pred_val))


Accuracy on validation set: 0.9006711409395973
Classification Report:
               precision    recall  f1-score   support

           0       0.80      0.75      0.77       131
           1       0.94      0.91      0.93       126
           2       0.92      0.94      0.93       488

   micro avg       0.90      0.90      0.90       745
   macro avg       0.89      0.87      0.88       745
weighted avg       0.90      0.90      0.90       745
 samples avg       0.90      0.90      0.90       745



## **Training and Evaluating a KNN Model on Processed Data**

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, accuracy_score

def knn_classifier(X_train, Y_train, X_test, n_neighbors=3):
    # Initialize KNN classifier
    knn = KNeighborsClassifier(n_neighbors=n_neighbors)

    # Fit the model
    knn.fit(X_train, Y_train)

    # Predict and return labels
    return knn.predict(X_test)

Y_pred_val_knn = knn_classifier(X_train_flat, Y_train, X_val_flat)
knn_accuracy = accuracy_score(Y_val, Y_pred_val_knn)
print("Accuracy on validation set:", accuracy_score(Y_val, Y_pred_val_knn))
print("Classification Report:\n", classification_report(Y_val, Y_pred_val_knn))


Accuracy on validation set: 0.7879194630872484
Classification Report:
               precision    recall  f1-score   support

           0       0.81      0.27      0.40       131
           1       0.96      0.60      0.74       126
           2       0.77      0.98      0.86       488

   micro avg       0.79      0.79      0.79       745
   macro avg       0.85      0.61      0.67       745
weighted avg       0.81      0.79      0.76       745
 samples avg       0.79      0.79      0.79       745



## **Model's Performance Summary**

In [31]:
model_summary = {
    'Model': ['Random Forest', 'Decision Tree', 'K-Nearest Neighbors'],
    'Accuracy': [rf_accuracy, dt_accuracy, knn_accuracy]
}

# Create a DataFrame to present the results
summary_df = pd.DataFrame(model_summary)

# Display the summary
summary_df

,Model,Accuracy
0,Random Forest,0.953020
1,Decision Tree,0.900671
2,K-Nearest Neighbors,0.787919


## **Statistical ANOVA Test on MFCC Coefficients**

In [ ]:
from scipy.stats import f_oneway, ttest_ind
import pandas as pd
import numpy as np

# Flatten the data
X_train_flat = np.array([x.flatten() for x in X_train])
Y_train_labels = np.argmax(Y_train, axis=1)

# Converting to DataFrame for easier manipulation
df = pd.DataFrame(X_train_flat)
df['label'] = Y_train_labels

# Selecting a specific MFCC coefficient to test, e.g., the first coefficient
mfcc_index = 0
grouped_data = [df[df['label'] == label][mfcc_index] for label in np.unique(Y_train_labels)]

# Performing ANOVA to test differences in means across groups
f_stat, p_value = f_oneway(*grouped_data)

print("ANOVA Results on MFCC Coefficients:")
print(f"F-statistic: {f_stat:.4f}, p-value: {p_value:.4f}")

# Interpretation
if p_value < 0.05:
    print("Significant differences exist in MFCC values across accent classes.")
else:
    print("No significant differences found in MFCC values across accent classes.")


ANOVA Results on MFCC Coefficients:
F-statistic: 0.4412, p-value: 0.6433
No significant differences found in MFCC values across accent classes.


## **Statistical T-Test for Comparison of Model Performance**

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from scipy.stats import ttest_rel
import numpy as np


#Training Random Forest
best_rf_model = RandomForestClassifier(n_estimators=200, random_state=42)
best_rf_model.fit(X_train_flat, Y_train)

#Training Decision Tree Model
best_dt_model = DecisionTreeClassifier(random_state=42)
best_dt_model.fit(X_train_flat, Y_train)

#Predicting on validation set
Y_pred_rf = best_rf_model.predict(X_val_flat)  # Predictions from Random Forest
Y_pred_dt = best_dt_model.predict(X_val_flat)  # Predictions from Decision Tree

#Computing correctness for paired t-test
rf_correct = (Y_pred_rf == Y_val).astype(int)  # 1 if correct, 0 otherwise
dt_correct = (Y_pred_dt == Y_val).astype(int)  # 1 if correct, 0 otherwise

#Performing paired t-test
t_statistic, p_value = ttest_rel(rf_correct, dt_correct)

#Displaying results
print("Paired t-test Results:")
print(f"T-statistic: {t_statistic[0]:.4f}, p-value: {p_value[0]:.4f}")

# Interpretation
if p_value[0] < 0.05:
    print("The difference in performance between Random Forest and Decision Tree is statistically significant.")
else:
    print("No significant difference in performance between Random Forest and Decision Tree.")

#Output of accuracies for context
rf_accuracy = accuracy_score(Y_val, Y_pred_rf)
dt_accuracy = accuracy_score(Y_val, Y_pred_dt)

print("\nModel Accuracies:")
print(f"Random Forest Accuracy: {rf_accuracy:.4%}")
print(f"Decision Tree Accuracy: {dt_accuracy:.4%}")


Paired t-test Results:
T-statistic: 4.2920, p-value: 0.0000
The difference in performance between Random Forest and Decision Tree is statistically significant.

Model Accuracies:
Random Forest Accuracy: 95.3020%
Decision Tree Accuracy: 89.2617%
